In [1]:
import numpy as np
import pandas as pd
import sys,os 
import random

os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse

sys.path.insert(0, os.path.join(os.path.abspath(''), '../../', 'magic'))

from tqdm import tqdm
from run_magic import run_magic_birw

In [2]:
def get_data_for_i(i):
    original_ = pd.read_csv('../../data/cell_simulation_10000/data.csv.gz', index_col=0)
    df_ = pd.read_csv('../../data/cell_simulation_10000/drp_{}0.csv.gz'.format(i), index_col=0)
    df_.index = [int(i) for i in df_.index]
    df_.columns = [int(i) for i in df_.columns]

    original_.columns = df_.columns
    original_.index = df_.index

    n = original_.size
    original_val = original_.values.copy()
    t = list(np.ndindex(original_.shape))
    random.Random(42).shuffle(t)

    mask = t[:int(len(t)/10 * i)]

    thr = np.sum(np.sign(df_)) > 0
    original_ = original_.loc[:, list(thr)]
    df_ = df_.loc[:, list(thr)]

    # original = original_.values
    original = np.log(original_+1)

    # df = df_.values
    df = np.log(df_+1)

    tmp = pd.DataFrame(thr)
    remove = [int(i) for i in tmp[tmp[0] == False].index]
    mask = [i for i in mask if i[1] not in remove]
    
    return df, mask, original

In [3]:
kuang_params = {  
    'is_preprocess': False,
    'is_pca': False,
    'Iter': 100, 
    'alpha': 0.1,
    'kernel': 'jp',
#     'method': 'cartesian',
}

In [4]:
mses = {}
corrs = {}
method = 'BiRW'

for i in (range(5)):
    print(i)
    df, mask, original = get_data_for_i(i+1)

    pred = run_magic_birw(
        df, 
        **kuang_params
    )
    pred = pd.DataFrame(pred, columns=df.columns, index=df.index)
    
    pred.to_csv('/export/scratch/inoue019/cell_result/{}_{}.csv.gz'.format(method, i), compression='gzip')

    origin = np.array([original.loc[i] for i in mask])
    predict = np.array([pred.loc[i] for i in mask])

    mses[i] = mse(origin, predict)
    corrs[i] = np.corrcoef(origin, predict)[0][1]

0
Running ...
     Time to compute cell-cell affinity matrix:  24.117613554000854
     Time to compute gene-gene affinity matrix:  7.032299518585205
Method: Tensor product


100%|██████████| 100/100 [00:14<00:00,  6.80it/s]


     Time to perform bi-random walk:  16.991628885269165
Finish running, Time:  48.578911781311035
1
Running ...
     Time to compute cell-cell affinity matrix:  31.697758197784424
     Time to compute gene-gene affinity matrix:  7.025343656539917
Method: Tensor product


100%|██████████| 100/100 [00:12<00:00,  8.07it/s]


     Time to perform bi-random walk:  12.624976873397827
Finish running, Time:  51.787635803222656
2
Running ...
     Time to compute cell-cell affinity matrix:  23.599912405014038
     Time to compute gene-gene affinity matrix:  6.959671974182129
Method: Tensor product


...

     Time to perform bi-random walk:  12.74049162864685
Finish running, Time:  43.73309683799744
3
Running ...
     Time to compute cell-cell affinity matrix:  23.341698169708252
     Time to compute gene-gene affinity matrix:  6.778822422027588
Method: Tensor product


100%|██████████| 100/100 [00:12<00:00,  8.00it/s]


     Time to perform bi-random walk:  12.719814777374268
Finish running, Time:  43.263848543167114
4
Running ...
     Time to compute cell-cell affinity matrix:  23.390260696411133
     Time to compute gene-gene affinity matrix:  6.8579206466674805
Method: Tensor product


100%|██████████| 100/100 [00:12<00:00,  8.07it/s]


     Time to perform bi-random walk:  12.632219552993774
Finish running, Time:  43.31411361694336


In [5]:
pd.DataFrame(mses.values()).T

,0,1,2,3,4
0,0.383481,0.451385,0.523747,0.588407,0.63835


In [6]:
pd.DataFrame(corrs.values()).T

,0,1,2,3,4
0,0.665391,0.651565,0.634533,0.612774,0.584


In [7]:
mses = {}
corrs = {}
method = 'BiRW'

for i in (range(5, 9)):
    print(i)
    df, mask, original = get_data_for_i(i+1)

    pred = run_magic_birw(
        df, 
        **kuang_params
    )
    pred = pd.DataFrame(pred, columns=df.columns, index=df.index)
    
    pred.to_csv('/export/scratch/inoue019/cell_result/{}_{}.csv.gz'.format(method, i), compression='gzip')

    origin = np.array([original.loc[i] for i in mask])
    predict = np.array([pred.loc[i] for i in mask])

    mses[i] = mse(origin, predict)
    corrs[i] = np.corrcoef(origin, predict)[0][1]

5
Running ...
     Time to compute cell-cell affinity matrix:  24.285328149795532
     Time to compute gene-gene affinity matrix:  6.704464435577393
Method: Tensor product


100%|██████████| 100/100 [00:12<00:00,  8.04it/s]


     Time to perform bi-random walk:  12.67353343963623
Finish running, Time:  44.0968132019043
6
Running ...
     Time to compute cell-cell affinity matrix:  25.324512481689453
     Time to compute gene-gene affinity matrix:  6.678128242492676
Method: Tensor product


100%|██████████| 100/100 [00:12<00:00,  8.07it/s]


     Time to perform bi-random walk:  12.629993438720703
Finish running, Time:  45.248709201812744
7
Running ...
     Time to compute cell-cell affinity matrix:  24.544204711914062
     Time to compute gene-gene affinity matrix:  7.101137638092041
Method: Tensor product


100%|██████████| 100/100 [00:12<00:00,  8.07it/s]


     Time to perform bi-random walk:  12.630648374557495
Finish running, Time:  44.898542642593384
8
Running ...
     Time to compute cell-cell affinity matrix:  23.533915519714355
     Time to compute gene-gene affinity matrix:  6.7291882038116455
Method: Tensor product


100%|██████████| 100/100 [00:12<00:00,  8.08it/s]


     Time to perform bi-random walk:  12.849055767059326
Finish running, Time:  43.72613215446472


In [8]:
pd.DataFrame(mses.values()).T

,0,1,2,3
0,0.673401,0.698358,0.716935,0.731025


In [9]:
pd.DataFrame(corrs.values()).T

,0,1,2,3
0,0.551871,0.512071,0.354844,0.15671
